## Individual runs from primary analysis
This notebook illustrates the trajectories of individual runs from the candidate ('mob') analysis.

### Running over Colab
Uncomment the commands in the following cell to install in Colab.
Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment and then proceed to the following cells.

### Generating documentation
Running the final cell will take several minutes and will generate the corresponding PDF documentation.
This can be downloaded from the Files section of the sidebar.

In [ ]:
# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.9 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch main
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
import pandas as pd
import numpy as np
from emutools.tex import StandardTexDoc, add_image_to_doc
import arviz as az
from inputs.constants import SUPPLEMENT_PATH, RUNS_PATH, RUN_IDS, BURN_IN
from aust_covid.calibration import get_targets
from emutools.calibration import plot_spaghetti, plot_param_hover_spaghetti
from aust_covid.plotting import plot_cdr_examples, plot_3d_spaghetti
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'spaghetti', 'Example calibration runs', 'austcovid')
targets = get_targets(app_doc, cross_ref=False)
spaghetti = pd.read_hdf(RUNS_PATH / RUN_IDS['mob'] / 'output/results.hdf', 'spaghetti')

In [ ]:
key_indicators_fig = plot_spaghetti(spaghetti, ['notifications_ma', 'deaths_ma', 'adult_seropos_prop', 'reproduction_number'], 2, targets)
key_outputs_title = 'Key outputs for randomly sampled runs from calibration algorithm.'
add_image_to_doc(key_indicators_fig.update_layout(showlegend=False), 'multioutput_spaghetti', 'svg', key_outputs_title, app_doc, 'Calibration results')
key_indicators_fig

In [ ]:
idata = az.from_netcdf(RUNS_PATH / RUN_IDS['mob'] / 'output/calib_full_out.nc')
burnt_idata = idata.sel(draw=np.s_[BURN_IN:])
n_samples = 12
sampled_idata = az.extract(burnt_idata, num_samples=n_samples)
cdr_caption = 'Examples of the modelled effect of various starting CDR proportion parameters.'
print(cdr_caption)
cdr_fig = plot_cdr_examples(sampled_idata.variables['start_cdr'])
caption = f'Modelled case detection ratio over time for {str(n_samples)} randomly selected parameter draws from calibration algorithm.'
add_image_to_doc(cdr_fig, 'cdr_examples', 'svg', cdr_caption, app_doc, 'Model construction', caption=caption)
cdr_fig

In [ ]:
plot_param_hover_spaghetti(spaghetti['notifications_ma'], idata)

In [ ]:
app_doc.write_doc()

In [ ]:
plot_3d_spaghetti('adult_seropos_prop', spaghetti, targets)

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y update
    ! apt-get -y install inkscape texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir(SUPPLEMENT_PATH)
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex -shell-escape spaghetti
    ! biber spaghetti
    ! pdflatex -shell-escape spaghetti
    ! pdflatex -shell-escape spaghetti